## Proyecto Final
Integrantes:
- Miguel Nieto
- Jhan Utria
- Camilo Ruiz
- Hermann Blanco

Universidad del Norte - Estructura del Computador II

### Descripción del contexto.

Visualización de Datos del dataset *indian_prison*, que contiene información sobre la población de prisioneros en la India de los años **2001 al 2013**.

La descripción de las columnas del dataset es la siguiente:

|Columna |	Descripción |  
|--------|--------------|
|state_ut_name|Estado en donde se llevó el censo |   
|year| Año del censo|   
|jail_type| Tipo de institución  | 
|year | Año de la observación|   
|male_convicts | Convictos masculinos|  
|female_convicts |  Convictos femeninos|  
|total_convicts|    Número total de convictos |  
|male_under_trial|  Hombres bajo juicio  | 
|female_under_trial| Mujeres bajo juicio | 
|total_under_trial| Total de personas bajo juicio|
|male_detenues| Detenidos masculinos|
|female_detenues|Detenidos femeninos|
|total_detenues| Total de detenidos|
|male_others| Ocupantes masculinos bajo otra razón|
|female_others| Ocupantes femeninos bajo otra razón|
|total_others|Número total de ocupantes bajo otra razón|
|total_male|Número total de ocupantes masculinos|
|total_female|Número total de ocupantes femeninos|
|grand_total| Número total de ocupantes|

In [103]:
import psycopg2
import numpy as np
import pandas as pd
import pandas.io.sql as psql
import plotly.express as px
import plotly.graph_objects as go

In [107]:
# establishing the connection
conn = psycopg2.connect(
   database='indian_prison_db', user='postgres', password='sa123456', host='postgresdb', port='5432'
)

df = psql.read_sql('SELECT * FROM indian_prison', conn, index_col='id')

conn.close()

In [108]:
df.head()

,state_ut_name,year,jail_type,male_convicts,female_convicts,total_convicts,male_under_trial,female_under_trial,total_under_trial,male_detenues,female_detenues,total_detenues,male_others,female_others,total_others,total_male,total_female,grand_total
id,,,,,,,,,,,,,,,,,,
0,Andhra Pradesh,2001,Central Jail,3372,3,3375,2718,114,2832,4,0,4,22,0,22,6116,117,6233
1,Andhra Pradesh,2001,District Jail,294,1,295,2379,136,2515,0,0,0,0,0,0,2673,137,2810
2,Andhra Pradesh,2001,Sub_Jail,188,10,198,3263,151,3414,0,0,0,0,0,0,3451,161,3612
3,Andhra Pradesh,2001,Women Jail,0,88,88,0,81,81,0,1,1,0,0,0,0,170,170
4,Andhra Pradesh,2001,Borstal School,17,0,17,0,0,0,0,0,0,0,0,0,17,0,17


### Grafica de personas por género, dependiendo del caso por el que estén en prisión.

In [109]:
x = ['Convictos', 'Bajo juicio', 'Detenidos', 'Otras razones']
male = [df['male_convicts'].sum(),
       df['male_under_trial'].sum(),
       df['male_detenues'].sum(),
       df['male_others'].sum()]

female = [df['female_convicts'].sum(),
         df['female_under_trial'].sum(),
         df['female_detenues'].sum(), 
         df['female_others'].sum()]

df_ = pd.DataFrame({'Másculino': male, 'Femenino': female})
px.bar(df_, x=x, y=['Másculino', 'Femenino'], barmode='group', labels={'variable': 'Sexo'})

En esta grafica se puede visualizar la diferencia entre los casos de *Bajo juicio* y *Convictos* frente a *Detenidos* y *Otras razones*, teniendo los primeros una cantidad superior. Cabe resaltar que la mayoría de personas están bajo juicio, lo que puede dar posibilidad a que la cantidad de personas en prisión reduzcan dependiendo del resultado del juicio.

**NOTA**: Tener en cuenta que los valores del eje Y están a una escala de *1e6 = $10^6$*.

### Grafica de el comportamiento de las personas en prisión, según el caso, a lo largo del tiempo.


In [116]:
casesbyyear = df.groupby('year').sum().reset_index()
casesbyyear

,year,male_convicts,female_convicts,total_convicts,male_under_trial,female_under_trial,total_under_trial,male_detenues,female_detenues,total_detenues,male_others,female_others,total_others,total_male,total_female,grand_total
0,2001,146948,4378,151326,427270,14364,441634,6476,544,7020,24388,2902,27290,605082,22188,627270
1,2002,158918,5324,164242,430554,15522,446076,9152,512,9664,21996,2736,24732,620620,24094,644714
2,2003,176792,6740,183532,419888,15428,435316,7682,334,8016,23116,3058,26174,627478,25560,653038
3,2004,190338,6716,197054,418556,15704,434260,8574,408,8982,19374,3112,22486,636842,25940,662782
4,2005,210058,7086,217144,456012,18140,474152,4918,166,5084,17776,2580,20356,688764,27972,716736
5,2006,225542,7808,233350,471578,18910,490488,4358,192,4550,15750,2404,18154,717228,29314,746542
6,2007,232114,8116,240230,479428,22026,501454,8834,540,9374,1614,120,1734,721990,30802,752792
7,2008,237828,8786,246614,493052,22804,515856,5772,184,5956,996,84,1080,737648,31858,769506
8,2009,238706,9176,247882,479034,21374,500408,4262,202,4464,1124,60,1184,723126,30812,753938
9,2010,242314,9264,251578,459692,20504,480196,4438,212,4650,1478,94,1572,707922,30074,737996


In [124]:

fig = go.Figure()
toplot = ['total_convicts', 'total_under_trial', 'total_detenues', 'total_others']

for col in toplot:
    fig.add_trace(go.Scatter(x=casesbyyear['year'],
                         y=casesbyyear[col],
                         mode='lines+markers',
                            name=col))
fig.show()


La cantifad de personas detenidas (*total_detenues*) en prisión ha mantenido un comportamiento constante a lo largo de los años mostrados [2001-2013], mientras que la cantidad de personas bajo juicio (*total_under_trial*) no siempre aumentan o disminuyen, es decir, su comportamiento es inestable, sin embargo, la tendencia general es ascendente pasando de tener aprox. 441634 personas en 2001 a 557000 en el año 2013. Por otro lado, la cantidad de personas que se encuentran en la carcel bajo otras razones (*total_others*) ha estado disminuyendo paulatinamente.

### Casos totales a lo largo del tiempo

In [121]:
totalbyyear = df.groupby(['year', 'state_ut_name']).sum().reset_index()
totalbyyear

,year,state_ut_name,male_convicts,female_convicts,total_convicts,male_under_trial,female_under_trial,total_under_trial,male_detenues,female_detenues,total_detenues,male_others,female_others,total_others,total_male,total_female,grand_total
0,2001,A & N Islands,428,6,434,330,2,332,0,0,0,0,0,0,758,8,766
1,2001,Andhra Pradesh,8238,204,8442,16720,964,17684,8,2,10,44,0,44,25010,1170,26180
2,2001,Assam,4712,94,4806,7876,254,8130,32,0,32,2,0,2,12622,348,12970
3,2001,Bihar,9974,156,10130,60000,1586,61586,24,0,24,74,0,74,70072,1742,71814
4,2001,Chandigarh,192,12,204,612,36,648,0,0,0,0,0,0,804,48,852
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
442,2013,Tamilnadu,8972,324,9296,15590,994,16584,3466,96,3562,0,0,0,28028,1414,29442
443,2013,Tripura,976,28,1004,776,22,798,0,0,0,0,0,0,1752,50,1802
444,2013,Uttar Pradesh,48694,1926,50620,111502,4698,116200,180,0,180,36,0,36,160412,6624,167036
445,2013,Uttaranchal,3672,170,3842,3654,194,3848,0,0,0,0,0,0,7326,364,7690


In [125]:
px.bar(totalbyyear, x='year', y='grand_total', color='state_ut_name', barmode='stack', title='Personas en prisión por año')

La tendencia general de la cantidad de personas en prisión es ascendente, aunque se puede notar que en el periodo del 2008 al 2010 se redujo el número de prisioneros en India. Otra información importante es que la mayor cantidad de esta población se ubica en el estado llamado **Uttar Pradesh**.

### Grafica del número de prisioneros por tipo de institución

In [100]:
totalbyjail_type = df[df.jail_type != 'Total'].groupby('jail_type')['grand_total'].sum().reset_index()
px.pie(totalbyjail_type, values='grand_total', names='jail_type', title='Número de prisoneros por tipo de institución')

Gracias a este gráfico es posible observar que la mayoría de personas en prisión están en los tipos de insticiones "Central Jail" y "District Jail". La diferencia que tienen estos dos contra el resto es bastante amplia.